In [ ]:
!python --version

Python 3.7.3


## Importing Dependencies

In [ ]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.metrics import plot_confusion_matrix 
from sklearn.metrics import accuracy_score

In [ ]:
!ls /datasets/localhardb

test.csv  train.csv


In [ ]:
df=pd.read_csv("/datasets/localhardb/train.csv")
df_val=pd.read_csv("/datasets/localhardb/test.csv")

In [ ]:
df.head(3)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING


In [ ]:
df.shape

(7352, 563)

In [ ]:
df.groupby('Activity')['Activity'].count()

Activity
LAYING                1407
SITTING               1286
STANDING              1374
WALKING               1226
WALKING_DOWNSTAIRS     986
WALKING_UPSTAIRS      1073
Name: Activity, dtype: int64

In [ ]:
target=df['Activity']

In [ ]:
df=df.drop('Activity',axis=1)

In [ ]:
target_val=df_val['Activity']
df_val=df_val.drop('Activity',axis=1)

## Normalizing The Data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
SS=StandardScaler()
df=SS.fit_transform(df)
df_val=SS.transform(df_val)

# Classifying with SVM

## Using GridSearchCV for optimizing hyperparameters

In [ ]:
parameter_pool = \
[
    {
        'kernel':['rbf','linear','sigmoid'],
        'C':[10,25,13],
        'gamma':[1e-3,3e-3,1e2]
    }
    ,
    {
        'kernel':['poly'],
        'C':[10,13,25],
        'degree':[2,3,4]
    }
]

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gscv=GridSearchCV(estimator=svm.SVC(),param_grid=parameter_pool,scoring='accuracy')

In [ ]:
gscv.fit(df,target)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [10, 25, 13], 'gamma': [0.001, 0.003, 100.0],
                          'kernel': ['rbf', 'linear', 'sigmoid']},
                         {'C': [10, 13, 25], 'degree': [2, 3, 4],
                          'kernel': ['poly']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
gscv.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'linear'}

In [ ]:
gscv.best_score_

0.9330881394025999

In [ ]:
SVM=svm.SVC(kernel='linear',C=9,gamma=0.000000001)
SVM.fit(df,target)
pred=SVM.predict(df_val)
accuracy_score(target_val,pred)

0.9589412962334578

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(target_val,pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.96      0.87      0.91       491
          STANDING       0.89      0.97      0.93       532
           WALKING       0.96      1.00      0.98       496
WALKING_DOWNSTAIRS       0.99      0.95      0.97       420
  WALKING_UPSTAIRS       0.97      0.96      0.96       471

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947

